Check that gradients are computed correctly 
(compare numeric derivative approximation computed using official C implementation and derivative computed by backpropagation using Pytorch version)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi']=200

import torch
import torch.nn.functional as F

from vmaf_torch import VMAF, VMAF_C, yuv_to_tensor, tensor_to_yuv

np.set_printoptions(precision=5, suppress=True)
torch.set_printoptions(precision=5, sci_mode=False)

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
# load reference yuv as tensors
#yuv_path_ref = "/storage/data/NFLX_dataset_public/ref/BigBuckBunny_25fps.yuv"
yuv_path_ref = "/storage/data/NFLX_dataset_public/ref/BirdsInCage_30fps.yuv"

width = 1920
height = 1080
num_frames = 1  

ref_y, ref_u, ref_v = yuv_to_tensor(yuv_path_ref, width, height, num_frames, color='yuv')
ref_y = ref_y.to(device)
ref_u = ref_u.to(device)
ref_v = ref_v.to(device)
ref_y.shape

torch.Size([1, 1, 1080, 1920])

In [4]:
# initialize VMAF Pytorch version
vmaf = VMAF(NEG=False, enable_motion=True, clip_score=True)
vmaf = vmaf.to(device)

In [5]:
# initialize VMAF C version
vmaf_c = VMAF_C(vmaf_executable='vmaf', vmaf_model_version='default')  # wrapper class for vmaf executable

In [6]:
#kernel_size = 3
kernel_size = 5

weight = torch.ones((1, 1, kernel_size, kernel_size))/kernel_size**2

weight = weight.to(device)
weight.requires_grad = True

weight

tensor([[[[0.04000, 0.04000, 0.04000, 0.04000, 0.04000],
          [0.04000, 0.04000, 0.04000, 0.04000, 0.04000],
          [0.04000, 0.04000, 0.04000, 0.04000, 0.04000],
          [0.04000, 0.04000, 0.04000, 0.04000, 0.04000],
          [0.04000, 0.04000, 0.04000, 0.04000, 0.04000]]]], device='cuda:0',
       requires_grad=True)

In [7]:
ref_y_conv = F.conv2d(ref_y, weight=weight, padding='same')

In [8]:
with torch.no_grad():
    print('Max pixel value:', ref_y_conv.max().item(), 'min pixel value:', ref_y_conv.min().item())

if ref_y_conv.max()>255 or ref_y_conv.min()<0:
    print('WARNING: images have pixels out of [0,255] range, the approximate gradients might be incorrect!')

Max pixel value: 230.2799835205078 min pixel value: 19.959999084472656


In [9]:
f = vmaf(ref_y, ref_y_conv)
print('f =', f.item())

f = 61.04484939575195


In [10]:
# compute derivatives via backpropagation
f.backward()
weight.grad

tensor([[[[148.16345, 168.11429, 179.23662, 177.40149, 162.86739],
          [166.76881, 188.79459, 199.59097, 194.09003, 174.89992],
          [176.37427, 198.75858, 207.17685, 199.56595, 177.43134],
          [174.69728, 194.16537, 200.52112, 190.49844, 168.83699],
          [163.28206, 178.01871, 180.62872, 170.31906, 150.94447]]]],
       device='cuda:0')

In [11]:
# select one element of the filter
coef_address = (0,0,kernel_size//2,kernel_size//2)
#coef_address = (0,0,1,2)
#coef_address = (0,0,0,0)

derivative = weight.grad[coef_address].item()

print("Derivative computed via backpropagation using Pytorch: ", derivative)

Derivative computed via backpropagation using Pytorch:  207.17684936523438


In [12]:
eps = 0.01

weight_plus_eps = weight.detach().clone()
weight_plus_eps[coef_address] += eps 

weight_minus_eps = weight.detach().clone()
weight_minus_eps[coef_address] -= eps 

weight_plus_eps, weight_minus_eps

(tensor([[[[0.04000, 0.04000, 0.04000, 0.04000, 0.04000],
           [0.04000, 0.04000, 0.04000, 0.04000, 0.04000],
           [0.04000, 0.04000, 0.05000, 0.04000, 0.04000],
           [0.04000, 0.04000, 0.04000, 0.04000, 0.04000],
           [0.04000, 0.04000, 0.04000, 0.04000, 0.04000]]]], device='cuda:0'),
 tensor([[[[0.04000, 0.04000, 0.04000, 0.04000, 0.04000],
           [0.04000, 0.04000, 0.04000, 0.04000, 0.04000],
           [0.04000, 0.04000, 0.03000, 0.04000, 0.04000],
           [0.04000, 0.04000, 0.04000, 0.04000, 0.04000],
           [0.04000, 0.04000, 0.04000, 0.04000, 0.04000]]]], device='cuda:0'))

In [13]:
ref_y_conv_plus_eps = F.conv2d(ref_y, weight=weight_plus_eps, padding='same')
ref_y_conv_minus_eps = F.conv2d(ref_y, weight=weight_minus_eps, padding='same')

In [14]:
print('Max pixel value:', ref_y_conv_plus_eps.max().item(), 'min pixel value:', ref_y_conv_plus_eps.min().item())
if ref_y_conv_plus_eps.max()>255 or ref_y_conv_plus_eps.min()<0:
    print('WARNING: images have pixels out of [0,255] range, the approximate gradients might be incorrect!')

Max pixel value: 232.58998107910156 min pixel value: 20.299999237060547


In [15]:
print('Max pixel value:', ref_y_conv_minus_eps.max().item(), 'min pixel value:', ref_y_conv_minus_eps.min().item())
if ref_y_conv_minus_eps.max()>255 or ref_y_conv_minus_eps.min()<0:
    print('WARNING: images have pixels out of [0,255] range, the approximate gradients might be incorrect!')

Max pixel value: 227.96998596191406 min pixel value: 19.619998931884766


In [16]:
# compute approximate derivative using reference C implementation
f_plus = vmaf_c.score_from_tensors((ref_y, ref_u, ref_v), (torch.round(ref_y_conv_plus_eps), ref_u, ref_v))
f_minus = vmaf_c.score_from_tensors((ref_y, ref_u, ref_v), (torch.round(ref_y_conv_minus_eps), ref_u, ref_v))
derivative_approx = (f_plus - f_minus) / (2 * eps)
print(f'{f_plus=}', f'{f_minus=}')

print("Derivative computed numerically using C implementation: ", derivative_approx)

Saving tensors to: ./ref.yuv and ./dist.yuv
Executing: vmaf -r ./ref.yuv -d ./dist.yuv -w 1920 -h 1080 --frame_cnt 1 -p 420 -b 8 --threads 16 -q --csv -o vmaf_out.csv 
Reading: vmaf_out.csv
Saving tensors to: ./ref.yuv and ./dist.yuv
Executing: vmaf -r ./ref.yuv -d ./dist.yuv -w 1920 -h 1080 --frame_cnt 1 -p 420 -b 8 --threads 16 -q --csv -o vmaf_out.csv 
Reading: vmaf_out.csv
f_plus=63.114191 f_minus=58.959366
Derivative computed numerically using C implementation:  207.74124999999975


In [17]:
print("Derivative computed numerically using C implementation: ", derivative_approx)
print("Derivative computed via backpropagation using Pytorch: ", derivative)
print("Difference", derivative-derivative_approx)

Derivative computed numerically using C implementation:  207.74124999999975
Derivative computed via backpropagation using Pytorch:  207.17684936523438
Difference -0.5644006347653772


In [18]:
# sanity check: compute approximate derivative using Pytorch VMAF implementation
#f_plus = vmaf(ref_y, ref_y_conv_plus_eps)
#f_minus = vmaf(ref_y, ref_y_conv_minus_eps)
#grad_approx = (f_plus - f_minus) / (2 * eps)
#print(f'{f_plus=}', f'{f_minus=}')
#print(f'{grad_approx=}')


In [19]:
# sanity check: compute approximate derivative using Pytorch VMAF implementation after rounding
#f_plus = vmaf(ref_y, torch.round(ref_y_conv_plus_eps))
#f_minus = vmaf(ref_y, torch.round(ref_y_conv_minus_eps))
#grad_approx = (f_plus - f_minus) / (2 * eps)
#print(f'{f_plus=}', f'{f_minus=}')
#print(f'{grad_approx=}')

In [20]:
# do all of this for every coefficient

vmaf_c.silent = True 

derivatives_c = []
derivatives_pytorch = []
dif = []

for i in range(kernel_size):
    for j in range(kernel_size):
        coef_address = (0,0,i,j)
        print('Coefficient:', i, j)
        
        derivative = weight.grad[coef_address].item()

        weight_plus_eps = weight.detach().clone()
        weight_plus_eps[coef_address] += eps 
        weight_minus_eps = weight.detach().clone()
        weight_minus_eps[coef_address] -= eps 

        ref_y_conv_plus_eps = F.conv2d(ref_y, weight=weight_plus_eps, padding='same')
        ref_y_conv_minus_eps = F.conv2d(ref_y, weight=weight_minus_eps, padding='same')

        if ref_y_conv_plus_eps.max()>255 or ref_y_conv_plus_eps.min()<0:
            print('WARNING: images have pixels out of [0,255] range, the approximate gradients might be incorrect!')
        if ref_y_conv_minus_eps.max()>255 or ref_y_conv_minus_eps.min()<0:
            print('WARNING: images have pixels out of [0,255] range, the approximate gradients might be incorrect!')
            
        # compute approximate derivative using reference C implementation
        f_plus = vmaf_c.score_from_tensors((ref_y, ref_u, ref_v), (torch.round(ref_y_conv_plus_eps), ref_u, ref_v))
        f_minus = vmaf_c.score_from_tensors((ref_y, ref_u, ref_v), (torch.round(ref_y_conv_minus_eps), ref_u, ref_v))
        derivative_approx = (f_plus - f_minus) / (2 * eps)

        print("Derivative computed numerically using C implementation: ", derivative_approx)
        print("Derivative computed via backpropagation using Pytorch: ", derivative)
        print("Difference", derivative-derivative_approx)
        
        derivatives_c.append(derivative_approx)
        derivatives_pytorch.append(derivative)
        dif.append(derivative-derivative_approx)

Coefficient: 0 0


Derivative computed numerically using C implementation:  147.5766
Derivative computed via backpropagation using Pytorch:  148.1634521484375
Difference 0.5868521484374867
Coefficient: 0 1
Derivative computed numerically using C implementation:  168.33855
Derivative computed via backpropagation using Pytorch:  168.11428833007812
Difference -0.2242616699218729
Coefficient: 0 2
Derivative computed numerically using C implementation:  180.22220000000004
Derivative computed via backpropagation using Pytorch:  179.2366180419922
Difference -0.985581958007856
Coefficient: 0 3
Derivative computed numerically using C implementation:  177.47409999999988
Derivative computed via backpropagation using Pytorch:  177.4014892578125
Difference -0.07261074218737917
Coefficient: 0 4
Derivative computed numerically using C implementation:  162.4544499999999
Derivative computed via backpropagation using Pytorch:  162.8673858642578
Difference 0.41293586425791773
Coefficient: 1 0
Derivative computed numericall

In [21]:
dif = np.abs(np.array(dif))
dif.mean(), dif.std()

(0.5739042109375305, 0.4530605698159361)